# RPC Batch API  Test
Use this notebook to send a small batch of RPC prediction requests to the deployed endpoint.

In [16]:
import json
import requests


RPC_ENDPOINT = "http://localhost:5000/predict/rpc"
LOGIN_ENDPOINT = RPC_ENDPOINT.replace("/predict/rpc", "/login")

#centralized login 
LOGIN_PAYLOAD = {
    "username": "admin",
    "password": "admin123",
}

# Maintain a session so we keep the auth cookie after login
session = requests.Session()

# Basic batch with representative callers
batch_instances = [
    {"name": "Excellent Customer", "features": [2.0, 0.98, 0.85]},
    {"name": "At-Risk Account", "features": [18.5, 0.42, 0.25]},
    {"name": "Average Profile", "features": [9.0, 0.61, 0.45]},
    # Custom scenario crafted to yield a very low contact probability (< 0.30)
    {"name": "Low Likelihood Caller", "features": [22.0, 0.30, 0.10]},
]

In [14]:
print(f"Logging in at {LOGIN_ENDPOINT} ...")
login_response = session.post(LOGIN_ENDPOINT, data=LOGIN_PAYLOAD, timeout=10)
print(f"Login HTTP {login_response.status_code}")
if login_response.status_code not in (200, 302):
    raise RuntimeError(f"Login failed: HTTP {login_response.status_code}")
final_url = login_response.url
if "login" in final_url.lower():
    raise RuntimeError("Login still on login page; check credentials.")
print("Login successful; session cookie stored.")

Logging in at http://localhost:5000/login ...
Login HTTP 200
Login successful; session cookie stored.
Login HTTP 200
Login successful; session cookie stored.


In [15]:
payload = {"instances": [row["features"] for row in batch_instances]}
print(f"Sending {len(payload['instances'])} instances to {RPC_ENDPOINT}")

try:
    response = session.post(RPC_ENDPOINT, json=payload, timeout=15)
    print(f"HTTP {response.status_code}")
    if response.history:
        chain = " -> ".join(str(r.status_code) for r in response.history)
        print(f"Redirect chain: {chain} -> {response.status_code}")
    content_type = response.headers.get("Content-Type", "")

    if "login" in response.url.lower():
        print("ERROR: Redirected back to login page. Check credentials or session handling.")
        preview = response.text.strip()
        if len(preview) > 400:
            preview = preview[:400] + '...'
        print(f"Response preview:\n{preview}")
    elif response.status_code == 200 and 'application/json' in content_type.lower():
        data = response.json()
        probs = data.get('probability', [])
        preds = data.get('predicted', [])
        print('Batch Results:')
        print('-' * 50)
        for idx, record in enumerate(batch_instances):
            prob = probs[idx] if idx < len(probs) else float('nan')
            pred = preds[idx] if idx < len(preds) else None
            label = 'Contact' if pred == 1 else 'No Contact'
            confidence = prob if pred == 1 else (1 - prob)
            print(f"{record['name']:<20} | {label:<10} | prob={prob:0.3f} | confidence={confidence:0.1%}")
    else:
        preview = response.text.strip()
        if len(preview) > 400:
            preview = preview[:400] + '...'
        print(f"Unexpected response (Content-Type={content_type}):\n{preview}")
except requests.RequestException as exc:
    print(f"Request failed: {exc}")

Sending 4 instances to http://localhost:5000/predict/rpc
HTTP 200
Batch Results:
--------------------------------------------------
Excellent Customer   | Contact    | prob=0.717 | confidence=71.7%
At-Risk Account      | Contact    | prob=0.521 | confidence=52.1%
Average Profile      | Contact    | prob=0.568 | confidence=56.8%
Low Likelihood Caller | No Contact | prob=0.437 | confidence=56.3%
HTTP 200
Batch Results:
--------------------------------------------------
Excellent Customer   | Contact    | prob=0.717 | confidence=71.7%
At-Risk Account      | Contact    | prob=0.521 | confidence=52.1%
Average Profile      | Contact    | prob=0.568 | confidence=56.8%
Low Likelihood Caller | No Contact | prob=0.437 | confidence=56.3%
